## カテゴリカル 変数をワンホットエンコ

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 200

import seaborn as sns
import pandas_profiling as pdp
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import collections
import re
import feather
import codecs

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold



In [94]:
train_feat6 = pd.read_feather('feature6/data/train_feat6_all.ftr')
test_feat6 = pd.read_feather('feature6/data/test_feat6_all.ftr')
df_all_feat6=pd.concat([train_feat6,test_feat6], axis=0, sort=False)

train_feat6_unique = pd.read_feather('feature6/data/train_feat6_unique.ftr')
test_feat6_unique = pd.read_feather('feature6/data/test_feat6_unique.ftr')
df_all=pd.concat([train_feat6_unique,test_feat6_unique], axis=0, sort=False)

In [95]:
def modify_null(df):

    # 部屋の階を最頻値で補完
    df['room_floor']=df['room_floor'].replace(-999,df.query('room_floor !=-999')['room_floor'].mode()[0])

    # 建物の階を最頻値で補完
    df['building_floor']=df['building_floor'].replace(-999,df.query('building_floor !=-999')['building_floor'].mode()[0])

    # 建物の全ての階を最頻値で補完
    df['total_floor']=df['total_floor'].replace(-999,df.query('total_floor !=-999')['total_floor'].mode()[0])

    # 駐車場料金を0で補完
    df['parking_price_car']=df['parking_price_car'].replace(-999,0)

    # 駐車場料金を0で補完
    df['parking_price_bicycle']=df['parking_price_bicycle'].replace(-999,0)

    # 駐車場料金を0で補完
    df['parking_price_bike']=df['parking_price_bike'].replace(-999,0)

    def is_null(x):
        if x==-999:
            return 1
        else:
            return 0

    # 駐車台数を０で補完、欠損かどうかの特徴を追加
    for col in ['parking_number_car','parking_number_bicycle','parking_number_bike']:
        df[col+'_isnull']=df[col].apply(lambda x:is_null(x))
        df[col]=df[col].replace(-999,0)

    #  駐車場の有無をワンホットエンコ
    dummy_cols=['is_parking_car','is_house_parking_car','is_other_parking_car',
                'is_parking_bicycle','is_house_parking_bicycle','is_other_parking_bicycle',
               'is_parking_bike','is_house_parking_bike','is_other_parking_bike']

    for col in dummy_cols:
        df[col]=df[col].astype(str)
        df=pd.get_dummies(df,columns=[col])

    # コンビニの数を最頻値で補完
    df['convenience_count']=df['convenience_count'].replace(-999,df.query('convenience_count !=-999')['convenience_count'].mode()[0])

    # スーパーの数を最頻値で補完
    df['supermarket_count']=df['supermarket_count'].replace(-999,df.query('supermarket_count !=-999')['supermarket_count'].mode()[0])

    # スーパーの数を最頻値で補完
    df['neighbor_count']=df['neighbor_count'].replace(-999,df.query('neighbor_count !=-999')['neighbor_count'].mode()[0])

    # 部屋の階と建物の階の積を平均で補完
    df['room_building_mul']=df['room_building_mul'].replace(-999,df.query('room_building_mul !=-999')['room_building_mul'].mean())

    # 面積のログと階の積を平均で補完
    df['square_log_building_mul']=df['square_log_building_mul'].replace(-999,df.query('square_log_building_mul !=-999')['square_log_building_mul'].mean())
    
    return df

In [96]:
df_all_feat7=modify_null(df_all_feat6)

## ＝＝＝＝＝保存＝＝＝＝＝

In [98]:
train_feat7=df_all_feat7[:train_index]
test_feat7=df_all_feat7[train_index:]

train_feat7.to_feather('feature7/data/train_feat7_all.ftr')
test_feat7.reset_index(drop=True,inplace=True)
test_feat7.to_feather('feature7/data/test_feat7_all.ftr')

In [107]:
train_feat6

,id,access_1_line,access_1_station,access_2_line,access_2_station,room_floor,building_floor,underground,total_floor,is_parking_car,is_house_parking_car,is_other_parking_car,parking_number_car,parking_price_car,is_parking_bicycle,is_house_parking_bicycle,is_other_parking_bicycle,parking_number_bicycle,parking_price_bicycle,is_parking_bike,is_house_parking_bike,is_other_parking_bike,parking_number_bike,parking_price_bike,convenience_count,supermarket_count,neighbor_count,デパート,幼稚園・保育園,図書館,ドラッグストア,郵便局,大学,公園,スーパー,銀行,コインパーキング,レンタルビデオ,飲食店,小学校,コンビニ,病院,総合病院,月極駐車場,クリーニング,学校,direction_enc,macro_structure,resistance_period,available_period,available_is_over,is_rent,period_num,future_available,future_available_is_over,floor_info_macro_structure,シャワー,トイレなし,トイレ別,バスなし,共同トイレ,共同バス,専用トイレ,専用バス,洗面台独立,浴室乾燥機,温水洗浄便座,脱衣所,追焚機能,ihコンロ,l字キッチン,カウンターキッチン,ガスコンロ,コンロ1口,コンロ2口,コンロ3口,コンロ4口以上,コンロ設置可,システムキッチン,冷蔵庫あり,口数不明,独立キッチン,給湯,電気コンロ,bsアンテナ,catv,csアンテナ,インターネット使用料無料,インターネット対応,光ファイバー,有線放送,高速インターネット,24時間換気システム,2面採光,3面採光,ウォークインクローゼット,エアコン付,エレベーター,オール電化,ガスその他,ガス暖房,クッションフロア,シューズボックス,タイル張り,トランクルーム,バリアフリー,バルコニー,フローリング,プロパンガス,ペアガラス,ルーフバルコニー,ロフト付き,下水,二世帯住宅,二重サッシ,井戸,公営水道,冷房,出窓,地下室,室内洗濯機置場,室外洗濯機置場,専用庭,床下収納,床暖房,排水その他,敷地内ごみ置き場,水道その他,汲み取り,洗濯機置場なし,浄化槽,石油暖房,都市ガス,防音室,longitude,latitude,address_city_enc,address_town_enc,R,K,DK,LDK,S,mesh_category_enc,rent_log,others_rent_log,square_num_log,access_1_distance_log,access_2_distance_log,old_num_log,Square/Room_log,room_number_log,nearest_distance,nearest_line,nearest_station,nearest_distance_log,is_top_room,room_building_mul,square_log_building_mul,Square_log/R,R_K_mul,R_DK_mul,R_LDK_mul,simple_room_sum,Square_log/simple_room_sum,Square_log/room_number,city_square_log_mean,city_squarelog_mean_diff
0,1,59,433,25,289,1,12,0,12,-999,-999,-999,-999,-999.0,1,1,0,1,-999.0,-999,-999,-999,-999,-999.0,1,2,10,0,0,0,956,1246,461,1103,311,0,0,0,378,495,588,0,0,0,0,0,5,4,564,446,0,0,24,422,0,96,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,35.746889,139.731307,3,1914,1,1,0,0,0,515,11.225257,0.000000,3.044999,5.771441,7.021976,4.779123,2.691746,0.897918,320.0,59,433,5.771441,0,12,36.539982,3.044999,1,0,0,2,1.522499,2.093520,3.384301,-0.339302
1,2,60,97,32,251,5,10,0,10,0,0,0,0,-999.0,1,1,0,1,-999.0,0,0,0,0,-999.0,0,1,1,0,0,0,0,0,0,0,1283,0,0,0,0,0,0,0,0,0,0,0,0,2,324,-215,1,0,24,-239,1,114,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,35.660879,139.778811,1,1368,1,0,0,0,0,715,11.238502,0.000000,2.862201,5.993961,6.580639,6.291569,2.862201,0.693147,400.0,60,97,5.993961,0,50,28.622009,2.862201,0,0,0,1,2.862201,2.862201,3.680058,-0.817857
2,3,17,383,17,80,12,15,0,15,1,0,1,1,30000.0,1,1,0,1,-999.0,1,1,0,1,-999.0,2,2,4,0,0,0,0,0,0,0,89,0,0,0,0,0,184,0,0,0,0,0,4,4,564,461,0,0,24,437,0,96,1,0,1,0,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,35.675199,139.666064,15,2179,1,1,0,0,0,147,11.608245,10.905956,3.137666,6.175867,6.329721,4.644391,2.782538,0.897918,480.0,17,383,6.175867,0,180,47.064987,3.137666,1,0,0,2,1.568833,2.157231,3.508224,-0.370558
3,4,54,497,6,235,3,4,0,4,0,0,0,0,-999.0,0,0,0,0,-999.0,0,0,0,0,-999.0,2,4,6,0,0,0,0,0,0,0,225,0,0,0,0,0,326,0,0,0,0,0,4,4,564,211,0,1,24,187,0,13,1,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,35.700157,139.650554,11,2824,2,0,0,1,0,85,11.918397,0.000000,4.118712,6.580639,5.484797,5.869297,2.846717,1.553116,240.0,6,235,5.484797,0,12,16.474848,2.059356,0,0,2,3,1.372904,1.105346,3.355182,0.763530
4,5,13,69,29,470,1,2,0,2,1,0,1,1,17000.0,0,0,0,0,-999.0,0,0,0,0,-999.0,1,4,5,0,0,0,0,0,0,0,193,0,0,0,0,0,351,0,0,0,0,0,4,1,264,-116,1,0,24,-140,1,119,1,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1

In [131]:
df=pd.read_csv('feature7/model/train.csv',index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198
0,1.349080,-1.116617,-0.008139,0.037420,0.277238,0.479203,0.01613,0.464317,-0.706874,-0.716141,0.45655,-0.065055,-0.442912,-0.108272,-1.938780,-0.977869,-2.511259,-0.103959,-0.155535,-0.178167,-0.308358,-0.276846,-0.149658,-0.271207,3.197090,-0.158525,-0.022551,-0.102131,-0.221826,-0.243,-0.739275,-0.290172,-0.186004,-0.006303,-0.058693,-0.189676,-1.926340,-0.868984,-0.997384,-2.087640,2.287583,-0.234886,-0.090415,-2.086714,2.144896,0.817552,0.337154,0.0,-2.292996,-0.068045,-0.051663,-0.032157,0.585104,-1.760071,-1.340357,-1.042833,0.888955,-0.805752,-0.782075,-0.406752,-0.097912,-0.365773,0.736162,-0.337665,-1.034239,-0.462936,-0.040884,-0.265728,0.729757,-0.149328,-0.124142,-0.376994,0.437916,-0.129162,-0.864922,-0.843009,-0.647805,-0.457231,0.912033,-1.041423,-0.046798,-0.293327,-0.714981,-0.516352,-0.185814,-0.372582,0.238908,0.971830,-0.13137,-0.022733,-0.006303,-0.112216,0.607267,1.136867,-0.127884,-0.070649,0.503673,0.394262,-0.182237,-0.138841,-0.100162,-0.200103,-1.712321,-0.010918,-0.04806,-0.006303,-1.683221,-0.44415,-0.207505,-0.015442,0.385471,-0.240391,-0.079986,-0.138245,-0.233753,-0.012608,0.936271,-0.239835,-0.008915,-0.045937,-0.017831,-0.006303,-2.081717,-0.043254,-0.578122,0.638799,-1.438494,-0.056294,-0.502460,-0.837161,-0.394732,-0.638299,-0.139138,0.331696,-0.615270,-1.344909,-0.351261,-0.359338,1.055393,0.037878,-1.272738,-0.488748,1.319643,-1.068529,-0.256677,-0.519869,0.017997,0.066796,-0.131536,-0.789254,-0.361195,-0.553252,-1.479088,1.731642,1.378097,2.232671,-1.851732,-0.684424,-0.704475,-0.866818,-0.684424,1.626196,-0.827790,-0.684424,1.028332,-0.492115,-0.684424,1.063306,-0.518142,-0.704475,-0.409005,0.951169,-0.704475,-0.409137,0.951320,-0.704475,0.704601,-0.008915,-0.866818,1.220760,-0.451631,-0.866818,1.206843,-0.442739,-0.866818,0.876630,-0.074521,-0.198468,-0.165752,-0.919275,-0.351188,1.135103,0.704999,-0.158712,-0.606570,-1.064014,-0.912032,-0.157675,-0.143529
1,-0.865042,0.807951,-0.731999,-1.051508,2.068910,1.246025,0.01613,1.220024,1.033618,1.228974,0.45655,-0.065055,2.169617,-0.108272,0.330275,0.118140,-0.670204,-0.103959,-0.155535,-0.178167,-0.308358,-0.276846,-0.149658,-0.271207,-0.570187,-0.158525,-0.022551,-0.102131,-0.221826,-0.243,-0.016269,-0.290172,-0.186004,-0.006303,-0.058693,-0.189676,-0.257756,0.770310,0.733821,0.835087,-0.437143,-0.234886,-0.090415,0.837256,-0.466223,0.301949,0.337154,0.0,0.436111,-0.068045,-0.051663,-0.032157,0.585104,0.568159,0.746070,0.958926,0.888955,1.241076,-0.782075,-0.406752,-0.097912,-0.365773,0.736162,-0.337665,0.965973,-0.462936,-0.040884,-0.265728,0.729757,-0.149328,-0.124142,-0.376994,0.437916,-0.129162,1.156174,-0.843009,1.543674,-0.457231,0.912033,0.960225,-0.046798,-0.293327,-0.714981,-0.516352,-0.185814,2.683974,0.238908,0.971830,-0.13137,-0.022733,-0.006303,-0.112216,0.607267,1.136867,-0.127884,-0.070649,0.503673,0.394262,-0.182237,-0.138841,-0.100162,-0.200103,0.584003,-0.010918,-0.04806,-0.006303,0.594099,-0.44415,-0.207505,-0.015442,0.385471,-0.240391,-0.079986,-0.138245,-0.233753,-0.012608,0.936271,-0.239835,-0.008915,-0.045937,-0.017831,-0.006303,0.480373,-0.043254,-0.322585,-0.868188,0.583681,0.922902,-0.502460,1.194514,-0.394732,-0.638299,-0.139138,-1.285228,1.538875,-0.733655,-0.081194,-0.812264,0.090328,-0.305371,-0.612307,-0.263608,-0.824415,0.833069,0.02